In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint, pformat, PrettyPrinter
from bson.son import SON
from bson.objectid import ObjectId
from os.path import exists
import json
from ipy_table import *

In [2]:
# Connect to database and create variables for the collections
client = MongoClient()
db = client.lobbyradar
entities = db.entities
relations = db.relations

print("Relations: %s Dokumente" % relations.count())

Relations: 32137 Dokumente


In [3]:
# Some format strings and helpers
def printCols(iterable, columns=5, spacing=20):                                                                                                                                                                             
    formatElement = u"{:<%s}" % spacing
    
    row_format =  formatElement * columns
    for chunk in zip(*[iter(iterable)]*columns):
        print(row_format.format(*chunk))
        
    # Splitting might be uneven and we could have some elements left, so print those too
    # The missing elements are those with an index in range of iterable % columns at the end 
    rest=iterable[len(iterable) - (len(iterable) % columns):]
    print( (formatElement * len(rest)).format(*rest) )


# The entities collection

## Beispielhaftes Dokument

In [4]:
first_relation_document = relations.find_one()
pprint(first_relation_document)

{u'_id': ObjectId('54bd3c968b934da063413717'),
 u'created': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
 u'data': [{u'desc': u'Verbindung',
            u'format': u'association',
            u'importer': u'created by lobbyliste importer',
            u'key': u'association',
            u'value': {u'position': u'Vorstand',
                       u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
                       u'type': u'executive'}}],
 u'entities': [ObjectId('54bd3c768b934da06340f4c5'),
               ObjectId('54bd3c768b934da06340f4c7')],
 u'importer': u'lobbyliste',
 u'tags': [],
 u'type': u'general',
 u'updated': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
 u'weight': 1}


In [5]:
# We need to find the tool variety before we can use it
VARIETY = "../variety/variety.js"
if not exists(VARIETY):
     print("variety.js script NICHT in %s gefunden. Ausversehen verschoben?" % VARIETY)
        
# Save database analysis as JSON for processing 
shell_capture = ! mongo --quiet lobbyradar --eval "var collection = 'relations', outputFormat='json'" {VARIETY}
variety_schema_json= json.loads(shell_capture.n)

# And print as ascii
! mongo --quiet lobbyradar --eval "var collection = 'relations', outputFormat='ascii'" {VARIETY}

+------------------------------------------------------------------------------------------+
| key                       | types                 | occurrences | percents               |
| ------------------------- | --------------------- | ----------- | ---------------------- |
| _id                       | ObjectId              |       32137 | 100.000000000000000000 |
| created                   | Date                  |       32137 | 100.000000000000000000 |
| data                      | Array                 |       32137 | 100.000000000000000000 |
| entities                  | Array                 |       32137 | 100.000000000000000000 |
| importer                  | String,null           |       32137 | 100.000000000000000000 |
| tags                      | Array                 |       32137 | 100.000000000000000000 |
| type                      | String                |       32137 | 100.000000000000000000 |
| updated                   | Date                  |       32137 | 10

# Als erstes müssen wir herausfinden welche Felder die Dokumente überhaupt haben und was wir uns ansehen müssen. 
### Dazu betrachten wir die Schlüssel aller Dokumenten-Felder (mit unendlicher Tiefe)

In [32]:
fields = []
for x in variety_schema_json:
    fields.append( {'key': x['_id']['key'], 'type': x['value']['types']} )
pprint(fields)

[{'key': u'_id', 'type': [u'ObjectId']},
 {'key': u'created', 'type': [u'Date']},
 {'key': u'data', 'type': [u'Array']},
 {'key': u'entities', 'type': [u'Array']},
 {'key': u'importer', 'type': [u'String', u'null']},
 {'key': u'tags', 'type': [u'Array']},
 {'key': u'type', 'type': [u'String']},
 {'key': u'updated', 'type': [u'Date']},
 {'key': u'weight', 'type': [u'Number']},
 {'key': u'data.XX.desc', 'type': [u'String']},
 {'key': u'data.XX.format', 'type': [u'String']},
 {'key': u'data.XX.key', 'type': [u'String']},
 {'key': u'data.XX.value', 'type': [u'Boolean', u'Object', u'String']},
 {'key': u'data.XX.importer', 'type': [u'String', u'null']},
 {'key': u'data.XX.value.sources', 'type': [u'Array']},
 {'key': u'data.XX.value.position', 'type': [u'String', u'null']},
 {'key': u'data.XX.value.type', 'type': [u'String']},
 {'key': u'data.XX.auto', 'type': [u'Boolean']},
 {'key': u'data.XX.created', 'type': [u'Date']},
 {'key': u'data.XX.id', 'type': [u'String']},
 {'key': u'data.XX.upd

Das sind einige. Um die Beziehungen zwischen den Entitäten richtig zu verstehen, müssen wir diese aber untersuchen.

## {{ fields[0]['key'] }}
Die Object-Id des Dokuments. Sagt nichts über die Relation aus.

## {{ fields[1]['key'] }}
Ist vom Typ {{fields[1]['type']}} und vermutlich einfach das Datum, an dem dieses Dokument erstellt wurde. Kann durch ein einfaches RDF Tripel später ausgedrückt werden.

In [33]:
first_relation_document.get(fields[1]['key'])

datetime.datetime(2015, 1, 19, 17, 19, 18, 521000)

## {{ fields[3]['key'] }}                                                                                                                                                                     
Die Entitäten, die über die Relation miteinander assoziiert sind. Leider können wir nicht davon ausgehen, dass die Relation immer von der Ersten zur Zweiten Entität gerichtet ist.

In [34]:
first_relation_document.get(fields[3]['key'])

[ObjectId('54bd3c768b934da06340f4c5'), ObjectId('54bd3c768b934da06340f4c7')]

Um die Relationen zu verstehen benötigen wir eine Funktion, die

In [353]:
def lookup_entity(entity_id, queryMask=None):
    return entities.find_one({ "_id": entity_id }, queryMask)
def lookup_relation(relation_id, queryMask=None):
    return relations.find_one({ "_id": relation_id }, queryMask)
    
def get_relation_entities(relation_id): 
    # Look up the entity ids of a relation with provided id
    relation = lookup_relation(relation_id, {'_id': 0, 'data.desc': 1, 'entities': 1})

    if not relation:
        return None
    
    # Find all those entities in the entities collection
    (entity1, entity2) = [ lookup_entity(e_id) for e_id in relation['entities'] ]
    
    pprint(relation['data'][0]['desc'] + ":")
    pprint("%s (%s)  -->  %s (%s)" % (entity1['name'], entity1['_id'], entity2['name'], entity2['_id']))

## {{ fields[4]['key'] }}                                                                                                                                                                      
Der Importer wird vermutlich die (Haupt-)quelle sein, aus der diese Information beschafft. So ist bei Angela Merkel "parteispenden" als importer angegeben, was auch in den Quellen auf der offiziellen Website des Lobbyradar aufgeführt wird.
Hier ist es schon interessant, die unterschiedlichen Werte zu sehen.

In [35]:
# Aggregation mit Gruppierung der Dokumente nach Feld "importer", zählen der Gruppen und aufsteigend danach sortieren
pipeline = [ 
    { "$group": {"_id": "$importer", "count": {"$sum": 1}} },
    { "$sort": {"count": -1} }
]

for row in db.relations.aggregate(pipeline, cursor={}):
    print("%s: (%s)" % (row['_id'], row['count']))

lobbyliste: (15810)
bundestag: (5315)
parteispenden: (3402)
None: (2273)
pr: (2193)
dax: (668)
db.clean: (629)
laender: (529)
kabinette: (433)
parteispenden13: (356)
seitenwechsler: (350)
thinktanks: (174)
parteispenden14: (5)


## {{ fields[5]['key'] }}                                                                                                                                                                      
Vom Typ {{ fields[5]['type'] }} von Strings. Die Tags könnten wichtig bei der Suche nach Relationen und Schemata für eine Ontologie sein, also lassen wir die möglichen Werte ausgeben. 

In [36]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
pipeline = [
    {"$unwind": "$tags"},
    {"$group": {"_id": "$tags", "count": {"$sum": 1}}},
    {"$sort": {"count": -1} }
]

tags = [ u"%s (%s)" % (row['_id'], row['count']) for row in db.relations.aggregate(pipeline, cursor={}) ]
printCols(tags, columns=5, spacing=30)

partei (3926)                 nebentaetigkeit (2552)        committee (1804)              pr-und-lobbyagentur (1151)    spende (1081)                 
parteispende (1081)           anwaltskanzlei (1047)         dax (675)                     mdb (629)                     laender (603)                 
kabinette (557)               aufsichtsrat (496)            pharma (466)                  seitenwechsler (281)          bank (245)                    
regierung (230)               kabinett (229)                ruestung (220)                vorstand (192)                fernsehrat (174)              
fernsehen (168)               auto (154)                    medien (80)                   spd (74)                      cdu (72)                      
öffentlich-rechtlich (72)     gesundheit (56)               verkehr (48)                  lebensmittel (46)             energie (46)                  
radio (42)                    parteilos (38)                VW (36)                       indu

## {{ fields[6]['key'] }}                                                                                                                                                                      
Type beschränkt sich auf zwei Werte, general und publication.

In [37]:
# Aggregation mit Gruppierung der Dokumente nach Feld "type", zählen der Gruppen und aufsteigend danach sortieren
pipeline = [ 
    { "$group": {"_id": "$type", "count": {"$sum": 1}} },
]

for typ in db.relations.aggregate(pipeline, cursor={}):
    print("%s: %s" % (typ['_id'], typ['count']))

publication: 1
general: 32136


Der Type 'publication' ist wohl eher zu vernachlässigen.

## {{ fields[7]['key'] }}                              
Vom Typ {{ fields[7]['type'] }} und speichert den Zeitpunkt des letzten Updates.

## {{ fields[8]['key'] }}                                                                                                                                                                      
Mir bisher noch schleierhaft. Auffallend ist aber, dass die meisten Relationen ein Gewicht von 1 besitzen.

In [38]:
pipeline = [ 
    { "$group": {"_id": "$weight", "count": {"$sum": 1}} },
]
for w in relations.aggregate(pipeline, cursor={}):
    print("weight %s: %s" % (w['_id'], w['count']))

weight 0: 2896
weight 1: 29241


## {{ fields[2]['key'] }}                                                                                                                                                                      
Auch Data ist ein Array, das sich allerdings noch deutlich weiter verschachtelt. 

In [39]:
printCols([ field['key'] for field in fields[9:] ], spacing=30)

data.XX.desc                  data.XX.format                data.XX.key                   data.XX.value                 data.XX.importer              
data.XX.value.sources         data.XX.value.position        data.XX.value.type            data.XX.auto                  data.XX.created               
data.XX.id                    data.XX.updated               data.XX.value.start           data.XX.value.desc            data.XX.value.end             
data.XX.value.amount          data.XX.value.year            data.XX.value.activity        data.XX.value.level           data.XX.value.periodical      
data.XX.value.place           data.XX.value.unsalaried      data.XX.value.start.year      data.XX.value.verified        data.XX.value.start.month     
data.XX.value.end.year        data.XX.value.start.day       data.XX.value.end.month       data.XX.value.end.day         data.XX.position              


## {{ fields[9]['key'] }}                                                                                                                                                                      
Der größte Teil der Relationen sind als "Verbindung" beschrieben, also sehr ungenau.

In [41]:
pipeline = make_pipeline(unwind="data", group="data.desc", sort=True)
for desc in relations.aggregate(pipeline, cursor={}):
    print("Desc: %s: %s" % (desc['_id'], desc['count']))

Desc: Verbindung: 26258
Desc: Parteispende: 9673
Desc: Angaben zur Nebentätigkeit: 2673
Desc: Quelle: 1
Desc: Verifiziert: 1
Desc: Position: 1


Beispiel für Verbindung:

In [11]:
pprint(relations.find_one({"data.desc": "Verbindung"}))

{u'_id': ObjectId('54bd3c968b934da063413717'),
 u'created': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
 u'data': [{u'desc': u'Verbindung',
            u'format': u'association',
            u'importer': u'created by lobbyliste importer',
            u'key': u'association',
            u'value': {u'position': u'Vorstand',
                       u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
                       u'type': u'executive'}}],
 u'entities': [ObjectId('54bd3c768b934da06340f4c5'),
               ObjectId('54bd3c768b934da06340f4c7')],
 u'importer': u'lobbyliste',
 u'tags': [],
 u'type': u'general',
 u'updated': datetime.datetime(2015, 1, 19, 17, 19, 18, 521000),
 u'weight': 1}


Beispiel für Parteispende:

In [7]:
pprint(relations.find_one({"data.desc": "Parteispende"}))

{u'_id': ObjectId('54c2a4f0fe6a42c82bbabb54'),
 u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 563000),
 u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
            u'desc': u'Parteispende',
            u'format': u'donation',
            u'id': u'8744c78b0d974e477285b33b874cbaddd1962043cea900f5c4f3094ebb590ccb',
            u'importer': u'created by parteispenden importer',
            u'key': u'donation',
            u'updated': datetime.datetime(2015, 1, 23, 19, 45, 52, 532000),
            u'value': {u'amount': 16872.57,
                       u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                       u'year': 1994}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 23, 19, 47, 9, 848000),
            u'desc': u'Parteispende',
            u'format': u'donation',
            u'id': u'8c5fe66733aba6f4143094f464a0ca759580be5b0775f3

## {{ fields[10]['key'] }}                                                                                                                                                                     
Und vom Format eine 'association', also auch lediglich eine 'Verbindung'. Daraus lässt sich leider kaum Informationen gewinnen.

In [42]:
pipeline = make_pipeline(unwind="data", group="data.format", sort=True)
for form in relations.aggregate(pipeline, cursor={}):
    print("Desc: %s: %s" % (form['_id'], form['count']))

Desc: association: 26258
Desc: donation: 9673
Desc: activity: 2673
Desc: bool: 1
Desc: link: 1
Desc: string: 1


## {{ fields[11]['key'] }}                                                                                                                                                                     
Wenn mir hier kein Fehler unterlaufen ist, scheint "Key" fast die gleichen Werte wie format zu besitzen

In [43]:
pipeline = make_pipeline(unwind="data", group="data.key", sort=True)
for key in relations.aggregate(pipeline, cursor={}):
    print("Desc: %s: %s" % (key['_id'], key['count']))

Desc: association: 26258
Desc: donation: 9673
Desc: activity: 2673
Desc: verified: 1
Desc: source: 1
Desc: position: 1


## {{ fields[13]['key'] }}                                                                                                                                                                     

In [55]:
pipeline = make_pipeline(unwind="data", group="data.importer", sort=True)
importer = [ "%s: %s" % (r['_id'], r['count']) for r in relations.aggregate(pipeline, cursor={}) ]
print("Importer:")
printCols(importer, columns=2, spacing=80)

Importer:
created by lobbyliste importer: 15843                                           created by parteispenden importer: 5483                                         
None: 5031                                                                      Parteispenden: 3093                                                             
bundestag: 3056                                                                 created by pr importer: 2151                                                    
Bundestag Spenden von Personen und Beiträge von Mandatsträgern: 905             created by dax importer: 670                                                    
db.clean: 629                                                                   laender: 546                                                                    
kabinette: 480                                                                  created by seitenwechsler importer: 349                                         
created by thinktank imp

## {{ fields[17]['key'] }}                                                                                                                                                                     

In [58]:
pipeline = make_pipeline(unwind="data", group="data.auto", sort=True)
for r in relations.aggregate(pipeline, cursor={}):
     print("%s: %s" % (r['_id'], r['count']))

None: 25317
True: 12346
False: 944


## {{ fields[18]['key'] }}                                                                                                                                                                     
Vom typ {{ fields[18]['type'] }}, also das Erstellungsdatum. Beispiel:

In [60]:
relations.find_one({ "data.created": {"$exists": 1} }, {'data.created':1})
#relations.find_one({ "data": {"$elemMatch": {"$in": [''] }} })

{u'_id': ObjectId('54bd3c978b934da063413e6d'),
 u'data': [{u'created': datetime.datetime(2015, 1, 30, 13, 16, 53, 6000)}, {}]}

## {{ fields[19]['key'] }}                                                                                                                                                                     

In [61]:
relations.find_one({ "data.id": {"$exists": 1} }, {'data.id':1})

{u'_id': ObjectId('54bd3c978b934da063413e6d'),
 u'data': [{u'id': u'bd5fff82726759130d968abda0494dd38ff13e2d66e432b18c6c8dab0961043f'},
  {}]}

## {{ fields[20]['key'] }}                                                                                                                                                                     

In [62]:
relations.find_one({ "data.updated": {"$exists": 1} }, {'data.updated':1})

{u'_id': ObjectId('54bd3c978b934da063413e6d'),
 u'data': [{u'updated': datetime.datetime(2015, 1, 30, 13, 16, 53, 6000)}, {}]}

# {{ fields[12]['key'] }}                                                                                                                                                                     

## {{ fields[14]['key'] }}                                                                                                                                                                     

In [78]:
result = relations.aggregate(make_pipeline(unwind='data', group='data.value.sources', sort=True))
printCols([ "%s: %s" % (p['_id'], p['count']) for p in result ], columns=1, spacing=60)

[u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf']: 15854
[u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json']: 8576
None: 3883                                                  
[u'https://docs.google.com/spreadsheets/d/1caESI467tBJ8uwv0RqjI-3AJYYcrdceJhfRPAtm2yXw/edit?usp=sharing']: 963
[u'https://lobbypedia.de/wiki/Seitenwechsler_in_Deutschland_im_%C3%9Cberblick']: 352
[u'http://www.freshfields.com/newsandinsights.aspx?language=de&region=germany&itemtype=News&LangId=1031&SearchRegion=Global&cbDeals=true']: 263
[]: 215                                                     
[u'http://www.hengeler.com']: 213                           
[u'http://www.cliffordchance.com']: 197                     
[u'http://www.bakermckenzie.com/']: 144                     
[u'http://linklaters.de/aktuelles/presse/pressemitteilungen.html']: 139
[u'https://docs.google.com/spreadsheets/d/1sVup987wHu_jozH-X0mU6XZIJsUTkUjNijT0fgoBA6

## {{ fields[15]['key'] }}                                                                                                                                                                     

In [76]:
result = relations.aggregate(make_pipeline(unwind='data', group='data.value.position', sort=True))
printCols([ "%s: %s" % (p['_id'], p['count']) for p in result ], columns=3, spacing=60)

Vorstand: 16054                                             None: 11394                                                 Mitglied: 2635                                              
Ordentliches Mitglied: 993                                  Stellvertretendes Mitglied: 838                             Arbeitsverhältnis: 837                                      
Aufsichtsratsmitglied: 459                                  Mitglied des Kuratoriums: 404                               Mitglied des Beirates: 231                                  
Mitglied des Aufsichtsrates: 204                            Mitglied des Vorstandes: 173                                Vorstandsmitglied: 161                                      
Staatssekretär: 122                                         Mitglied im Rundfunkrat: 115                                Mitglied des Kreistages: 111                                
Parlamentarischer Staatssekretär: 87                        Mitglied des Aufsichtsrats: 84     

## {{ fields[16]['key'] }}                                                                                                                                                                     

In [81]:
result = relations.aggregate(make_pipeline(unwind='data', group='data.value.type', sort=True))
printCols([ "%s: %s" % (p['_id'], p['count']) for p in result ], columns=2, spacing=40)

executive: 17212                        None: 12338                             
member: 5129                            job: 1598                               
business: 1445                          government: 633                         
subsidiary: 82                          : 64                                    
sponsoring: 43                          committee: 28                           
pass: 19                                participant: 16                         


## {{ fields[21]['key'] }}                                                                                                                                                                     
Start ist ein Zeitpunkt, zu dem ein Verhältnis zweier Entitäten begonnen hat.

In [86]:
relations.find_one({ "data.value.start": {"$exists": 1, "$ne": None} }, {'data.value.start':1})

{u'_id': ObjectId('54bd3c988b934da0634141b5'),
 u'data': [{u'value': {u'start': {u'day': 1, u'month': 6, u'year': 2011}}}]}

Hier zum Beispiel eine Position als Vorstand bei einem Unternehmen.

In [89]:
relations.find_one({ "data.value.start": {"$exists": 1, "$ne": None} }, {'data':1})

{u'_id': ObjectId('54bd3c988b934da0634141b5'),
 u'data': [{u'desc': u'Verbindung',
   u'format': u'association',
   u'importer': u'created by lobbyliste importer',
   u'key': u'association',
   u'value': {u'position': u'Vorstand',
    u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
    u'start': {u'day': 1, u'month': 6, u'year': 2011},
    u'type': u'executive'}}]}

## {{ fields[22]['key'] }}                                                                                                                                                                     

In [94]:
descs = [ "%s: %s" % (d['_id'], d['count']) for d in relations.aggregate(make_pipeline(unwind='data', group='data.value.desc', sort=True))]
printCols(descs, columns=1, spacing=100)

None: 34325                                                                                         
Geschäftsverbindung: 1435                                                                           
Funktionen in Vereinen, Verbänden und Stiftungen: 1220                                              
Funktionen in Körperschaften und Anstalten des öffentlichen Rechts: 682                             
Funktionen in Unternehmen: 468                                                                      
Entgeltliche Tätigkeiten neben dem Mandat: 298                                                      
Bundestag Spenden von Personen und Beiträge von Mandatsträgern: 58                                  
Bündnis90/Die Grünen: 13                                                                            
Mitglied im Fernsehrat als Vertreterin aus den Bereichen des Erziehungs- und Bildungswesens, der Wissenschaft, der Kunst, der Kultur, der Filmwirtschaft, der Freien Berufe, der Familienarbeit,

In [90]:
relations.find_one({ "data.value.desc": {"$exists": 1, "$ne": None} }, {'data.value.desc':1})

{u'_id': ObjectId('54bd3c9b8b934da06341579f'),
 u'data': [{u'value': {u'desc': u'Funktionen in Vereinen, Verb\xe4nden und Stiftungen'}}]}

## {{ fields[23]['key'] }}                                                                                                                                                                     

In [95]:
relations.find_one({ "data.value.end": {"$exists": 1, "$ne": None} }, {'data.value.end':1})

{u'_id': ObjectId('54bd3c968b934da0634137ea'),
 u'data': [{u'value': {u'end': {u'day': 1, u'month': 5, u'year': 2015}}}]}

In [96]:
relations.find_one({ "data.value.end": {"$exists": 1, "$ne": None} }, {'data':1})

{u'_id': ObjectId('54bd3c968b934da0634137ea'),
 u'data': [{u'desc': u'Verbindung',
   u'format': u'association',
   u'importer': u'created by lobbyliste importer',
   u'key': u'association',
   u'value': {u'end': {u'day': 1, u'month': 5, u'year': 2015},
    u'position': u'Vorstand',
    u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
    u'type': u'executive'}}]}

## {{ fields[24]['key'] }}                                                                                                                                                                     
Das hier scheinen Nebeneinkünfte oder Parteispenden zu sein! Wichtig und auf jeden Fall relevant.

In [105]:
amounts = relations.find({ "data.value.amount": {"$exists": 1}}, {'data.value': 1}).limit(2)
pprint(list(amounts))

[{u'_id': ObjectId('54c2a4f0fe6a42c82bbabb54'),
  u'data': [{u'value': {u'amount': 16872.57,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 1994}},
            {u'value': {u'amount': 12500,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 2004}}]},
 {u'_id': ObjectId('54c2a4f0fe6a42c82bbabb55'),
  u'data': [{u'value': {u'amount': 15338.7,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 1994}},
            {u'value': {u'amount': 15000,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 2003}},
            {u'value': {u'amount': 15000,
                       

## {{ fields[25]['key'] }} 
Das Jahr einer Parteispende ist festgehalten zusammen mit einem Quellenbeleg.

In [107]:
relations.find_one({ "data.value.year": {"$exists": 1}}, {'data.value': 1})

{u'_id': ObjectId('54c2a4f0fe6a42c82bbabb54'),
 u'data': [{u'value': {u'amount': 16872.57,
    u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
    u'year': 1994}},
  {u'value': {u'amount': 12500,
    u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
    u'year': 2004}}]}

## {{ fields[26]['key'] }}                                                                                                                                                                     

In [112]:
printCols([ "%s: %s" % (a['_id'], a['count']) for a in relations.aggregate(make_pipeline(unwind="data", group="data.value.activity", sort=True))], columns=3, spacing=60)

None: 38418                                                 Rechtsanwalt: 62                                            Vortrag: 33                                                 
Betriebsleiter: 8                                           Beratung: 7                                                 Rechtsanwältin: 4                                           
Rechtsanwalt und Notar: 3                                   Wissenschaftliche Beratung: 3                               Publizistische Tätigkeit: 3                                 
Autorin: 2                                                  Betreuerin: 2                                               Landwirt/Geschäftsführer: 2                                 
Wirtschaftsprüfer/Steuerberater: 2                          Landwirt: 2                                                 Berater, freiberuflich: 2                                   
Lehrbeauftragte: 1                                          Kunde 10: 1                        

In [222]:
find_one_if_exists = lambda field, mask: relations.find_one({ field: {"$exists": 1, "$ne": None} }, {mask: 1})

## {{ fields[27]['key'] }}                                                                                                                                                                     

In [117]:
find_one_if_exists("data.value.level", "data.value")

{u'_id': ObjectId('54bd3c978b934da063413e6d'),
 u'data': [{u'value': {u'activity': None,
    u'desc': u'Funktionen in Vereinen, Verb\xe4nden und Stiftungen',
    u'end': None,
    u'level': 0,
    u'periodical': u'einmalig',
    u'place': None,
    u'position': u'Vorsitzende',
    u'start': None,
    u'unsalaried': True}},
  {u'value': {u'position': u'Vorstand',
    u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
    u'type': u'executive'}}]}

## {{ fields[28]['key'] }}                                                                                                                                                                     
Die Häufigkeit einer Beschäftigung oder ähnliches. Könnte auch hilfreich sein.

In [214]:
periodicals = relations.aggregate(make_pipeline(unwind='data', group='data.value.periodical', sort=True))
printCols( [ "%s: %s" % (d['_id'], d['count']) for d in periodicals ] )

None: 35934         einmalig: 2535      monatlich: 96       jährlich: 42        


Anscheinend bestand dieses Verhältnis nur einmalig, war eine Funktion in einem Verein bei der eine Person die Rolle des Vorsitzenden eingenommen hat und dabei nicht bezahlt wurde

In [223]:
pprint(find_one_if_exists("data.value.periodical", "data.value"))

{u'_id': ObjectId('54bd3c9b8b934da06341579f'),
 u'data': [{u'value': {u'activity': None,
                       u'desc': u'Funktionen in Vereinen, Verb\xe4nden und Stiftungen',
                       u'end': {},
                       u'level': 0,
                       u'periodical': u'einmalig',
                       u'place': None,
                       u'position': u'Mitglied des Vorstandes',
                       u'start': {u'month': 12, u'year': 2013},
                       u'type': u'executive',
                       u'unsalaried': True,
                       u'verified': True}}]}


## {{ fields[29]['key'] }}                                                                                                                                                                     
Ein Ort bzw eine Region, an dem ein (Arbeits-)verhältnis stattgefunden hat. Wird anscheinend nicht häufig verwendet.

In [232]:
find_one_if_exists("data.value.place", "data.value") 

{u'_id': ObjectId('54cb8443bbde97945a06a608'),
 u'data': [{u'value': {u'activity': u'Landwirt',
    u'desc': u'Entgeltliche T\xe4tigkeiten neben dem Mandat',
    u'end': {u'year': 2013},
    u'level': 3,
    u'periodical': u'einmalig',
    u'place': u'Oberhochstatt',
    u'position': None,
    u'start': {u'year': 2013},
    u'unsalaried': False}}]}

In [233]:
places = relations.aggregate(make_pipeline(unwind='data', group='data.value.place', sort=True))
printCols( [ "%s: %s" % (d['_id'], d['count']) for d in places ], spacing=30, columns=4)

None: 38591                   Bad Zwischenahn: 2            Emlichheim: 2                 Legden: 2                     
Vierlinden: 2                 Oberhochstatt: 2              Coburg: 1                     Haspa-Branchen Treff „Wirtschaftsfaktor Russland", Hamburg: 1
Lingen: 1                     Astellas GmbH, München: 1     Laar: 1                       Deutscher Zigarettenverband e.V., Berlin: 1



In [258]:
def make_pipeline(**kwargs):
    unwind = lambda field: {"$unwind": "$"+field}
    group = lambda field: {"$group": {"_id": "$"+field, "count": {"$sum": 1}} }
    sort = {"$sort": {"count": -1} }
    pipeline = []
    if (kwargs.has_key('unwind')): pipeline.append(unwind(kwargs['unwind']))
    if (kwargs.has_key('group')): pipeline.append(group(kwargs['group']))
    if (kwargs.has_key('sort')): pipeline.append(sort)
    return pipeline

# Returns a simple aggregationd directly as list of values
def simple_aggregate(group="", unwind=None, sort=True):
    pipeline = make_pipeline(unwind=unwind, group=group, sort=sort)
    return [ "%s: %s" % (d['_id'], d['count']) for d in relations.aggregate(pipeline, cursor={}) ]

## {{ fields[30]['key'] }}                                                                                                                                                                     
Ob eine Tätigkeit ehremamtlich stattgefunden hat. Scheinen ja nur wenige zugegeben zu haben ;)

In [251]:
unsalaried = simple_aggregate(unwind="data", group="data.value.unsalaried", sort=True)
printCols(unsalaried, columns=1)

None: 35934         
False: 2091         
True: 582           



## {{ fields[32]['key'] }}                                                                                                                                                                     

In [257]:
simple_aggregate(unwind="data", group="data.value.verified")

['None: 37524', 'True: 1083']

In [259]:
find_one_if_exists("data.value.verified", "data.value") 

{u'_id': ObjectId('54bd3c9b8b934da063415516'),
 u'data': [{u'value': {u'end': {u'year': 2014},
    u'position': u'Vize-Pr\xe4sident',
    u'sources': [u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'],
    u'start': {u'year': 2006},
    u'type': u'executive',
    u'verified': True}}]}

## {{ fields[35]['key'] }} | {{ fields[33]['key'] }} | {{ fields[31]['key'] }}

In [270]:
print(find_one_if_exists("data.value.start.day", "data.value.start")) 
print(find_one_if_exists("data.value.start.year", "data.value.start"))
print(find_one_if_exists("data.value.start.month", "data.value.start")) 

{u'_id': ObjectId('54bd3c988b934da0634141b5'), u'data': [{u'value': {u'start': {u'year': 2011, u'day': 1, u'month': 6}}}]}
{u'_id': ObjectId('54bd3c988b934da0634141b5'), u'data': [{u'value': {u'start': {u'year': 2011, u'day': 1, u'month': 6}}}]}
{u'_id': ObjectId('54bd3c988b934da0634141b5'), u'data': [{u'value': {u'start': {u'year': 2011, u'day': 1, u'month': 6}}}]}


## {{ fields[37]['key'] }} | {{ fields[36]['key'] }} | {{ fields[34]['key'] }} 

In [271]:
print(find_one_if_exists("data.value.end.day", "data.value.end"))
print(find_one_if_exists("data.value.end.month", "data.value.end"))
print(find_one_if_exists("data.value.end.year", "data.value.end")) 

{u'_id': ObjectId('54bd3c968b934da0634137ea'), u'data': [{u'value': {u'end': {u'year': 2015, u'day': 1, u'month': 5}}}]}
{u'_id': ObjectId('54bd3c968b934da0634137ea'), u'data': [{u'value': {u'end': {u'year': 2015, u'day': 1, u'month': 5}}}]}
{u'_id': ObjectId('54bd3c968b934da0634137ea'), u'data': [{u'value': {u'end': {u'year': 2015, u'day': 1, u'month': 5}}}]}


## {{ fields[38]['key'] }} 
Aaaaaaah ja...

In [272]:
printCols(simple_aggregate(unwind="data", group="data.position"))

None: 38606         Lobbyist: 1         


## {{ fields[39]['key'] }} 

In [277]:
printCols(simple_aggregate(unwind="data", group="data.value.url"))

None: 38606         https://lobbypedia.de/wiki/Zentrum_f%C3%BCr_Europ%C3%A4ische_Wirtschaftsforschung: 1


# Altes

In [288]:
# Werte von "Desc"
pipeline = [
    {"$unwind": "$data"},
    {"$group": {"_id": {"desc": "$data.desc", "format": "$data.format"}, "count": {"$sum": 1}}},
    {"$sort": {"count": -1} }
]

results = relations.aggregate(pipeline, cursor={})
printCols([ "%s/%s: %s" % (row['_id']['desc'], row['_id']['format'], row['count']) for row in results ], columns=1)

Verbindung/association: 26258
Parteispende/donation: 9673
Angaben zur Nebentätigkeit/activity: 2673
Verifiziert/bool: 1 
Quelle/link: 1      
Position/string: 1  



In [291]:
# Print an example value of each description type
for desc in distinct_desc_types:
    desc_type = desc['_id']['desc']
    entity_with_desc_type = db.relations.find_one({ "data.desc": {"$eq": desc_type} }, { "_id": 0, "data.desc": 1, "data.format": 1, "data.value.position": 1, "data.value.type": 1, "data.value.desc": 1 })
    #entity_with_desc_type = db.entities.find_one({ "data.desc": {"$eq": desc_type} }, { "data": {"$elemMatch": {"data.desc": desc_type}} })
    pprint(entity_with_desc_type)

{u'data': [{u'desc': u'Verbindung',
            u'format': u'association',
            u'value': {u'position': u'Vorstand', u'type': u'executive'}}]}
{u'data': [{u'desc': u'Parteispende', u'format': u'donation', u'value': {}},
           {u'desc': u'Parteispende', u'format': u'donation', u'value': {}}]}
{u'data': [{u'desc': u'Angaben zur Nebent\xe4tigkeit',
            u'format': u'activity',
            u'value': {u'desc': u'Funktionen in Vereinen, Verb\xe4nden und Stiftungen',
                       u'position': u'Vorsitzende'}},
           {u'desc': u'Verbindung',
            u'format': u'association',
            u'value': {u'position': u'Vorstand', u'type': u'executive'}}]}
{u'data': [{u'desc': u'Verbindung',
            u'format': u'association',
            u'value': {u'position': u'Vorstand', u'type': u'executive'}}]}
{u'data': [{u'desc': u'Angaben zur Nebent\xe4tigkeit',
            u'format': u'activity',
            u'value': {u'desc': u'Funktionen in Vereinen, Verb\xe4nden 